In [165]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from scipy import stats
import datetime

In [166]:
#Importing Files
#37 and 39 refer to the location, 37 is Easter Island, and 39 is a nearby seamount called Pukao.
#The Hab files are the habitat analysis that I did, CTD contains the CTD data by timecode, and DT contains the latitude and longitude data by timecode.
Hab37 = pd.read_csv('37 - Habitat Analysis.csv')
Hab39 = pd.read_csv('39 - Habitat Analysis.csv')
CTD37 = pd.read_csv('37_CTD.csv')
CTD39 = pd.read_csv('39_CTD.csv')
DT37 = pd.read_csv('37_DT.csv')
DT39 = pd.read_csv('39_DT.csv')

2 research sites - 37 (SPG6, a seamount) and 39 (SPG7, Easter Island). The Hab37 and Hab39 dataframes contain the habitat analysis - substrate type, texture, bioturbation, etc. CTD37 and CTD39 are the dataframes with the CTD data, and DT37 and DT39 contain the latitude, longitude data and depth position of the deeptow camera unit at each point.

In [167]:
#Looking at the file type
Hab37?

Type:        DataFrame
String form:
Image       DATE      TIME  \
           0   EPIC_Cruise_MR1 <...>         NaN
           87          NaN         NaN
           88          NaN         NaN
           
           [89 rows x 20 columns]
Length:      89
File:        c:\users\kara\anaconda3\lib\site-packages\pandas\core\frame.py
Docstring:  
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns).
Arithmetic operations align on both row and column labels. Can be
thought of as a dict-like container for Series objects. The primary
pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, or list-like objects.

    .. versionchanged:: 0.23.0
       If data is a dict, column order follows insertion-order for
       Python 3.6 and later.

    .. versionchanged:: 0.25.0
       If data is a list of dicts, colu

In [168]:
#Looking at the data
Hab37

,Image,DATE,TIME,"Subsystem (Seamount vs island, seamount top, flank, base, island slope upper <200, intermediate500-1000, lower 1000+)",Class (meso-macrohab - based on seafloor morphology),"Substrate subclass (organic debris, mud, sand, gravel, pebble, cobble, boulder, mixed sediment, bedrock, sedimentary, Tapia: rock, silty sediments, mixed substrate, coarse sand)","Slope subclass(flat 0-5, sloping 5-30, steeply sloping 30-60, vertical 60-90, overhang >90)","M - morphology (regular: homogeneous, little relief; irregular: non-uniform, relief 1-10 m; hummocky: uniform bottom with mounds or depressions 0-3 m; outcrops)","M - deposition (dusting <1 cm, thin 1-5 cm, thick > 5 cm)","M - texture (sorting, packing, density: occasional, scattered, contiguous, pavement)",M - physical processes,M - chemical processes,"M - biological processes (bioturbation, % cover, communities)",M - anthropogenic processes,Altitude,CPC?,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:10:29,Seamount - EI (SPG7),Flat sandy bottom,coarse sand,NaN,regular,NaN,NaN,2D ripples,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
1,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:11:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,4.2,NaN,NaN,NaN,NaN,NaN
2,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:12:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,6.8,NaN,NaN,NaN,NaN,NaN
3,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:13:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - dusting,NaN,NaN,NaN,NaN,NaN,6.8,Y,NaN,NaN,NaN,NaN
4,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:14:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - thin,NaN,NaN,NaN,NaN,NaN,10.3,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:39:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,urchins,NaN,3.0,NaN,NaN,NaN,NaN,NaN
85,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:40:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,"urchins, fish",NaN,3.3,NaN,NaN,NaN,NaN,NaN
86,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:41:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,"urchins, fish",NaN,3.4,NaN,NaN,NaN,NaN,NaN
87,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:42:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,urchins,NaN,2.9,NaN,NaN,NaN,NaN,NaN


In [169]:
CTD37

,DATE,TIME,ALT,CTD_COND,CTD_TEMP,CTD_DEPTH,CTD_SAL,HEAD(GYRO),ROLL(GYRO),PITCH(GYRO)
0,2/13/2019,19:42:49,---.-,5.21224,24.8125,14.16,34.4356,1.7,-1.1,3.1
1,2/13/2019,19:42:50,---.-,5.20695,24.8138,13.39,34.3955,358.8,-0.3,4.1
2,2/13/2019,19:42:51,---.-,5.23634,24.8127,13.25,34.6148,0.1,-1.5,6.7
3,2/13/2019,19:42:52,---.-,5.27145,24.8117,13.58,34.8766,0.1,-1.5,6.7
4,2/13/2019,19:42:53,---.-,5.23748,24.8128,14.14,34.6228,359.4,-1.2,9.5
...,...,...,...,...,...,...,...,...,...,...
7809,2/13/2019,21:52:58,---.-,5.46090,24.8806,14.63,36.2353,17.6,-0.9,9.6
7810,2/13/2019,21:52:59,---.-,5.46052,24.8773,14.97,36.2349,18.1,0.0,5.4
7811,2/13/2019,21:53:00,---.-,5.46184,24.8861,15.28,36.2377,18.4,0.9,0.4
7812,2/13/2019,21:53:01,---.-,5.46234,24.8902,15.12,36.2381,18.4,1.0,1.3


In [170]:
DT39

,date,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Latitude,Longitude,Depth,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,19:36:42,T1,1.0,X=8.6,Y=-200.7,27-04.0653 S,109-10.0314 W,Z=22.6,HR=88.0,BRG=178.9,SR=90.8
1,19:39:12,T1,1.0,X=29.1,Y=-194.1,27-04.0542 S,109-10.0274 W,Z=11.6,HR=90.2,BRG=178.7,SR=91.0
2,19:39:24,T1,1.0,X=28.3,Y=-194.9,27-04.0547 S,109-10.0279 W,Z=23.9,HR=89.1,BRG=179.0,SR=92.2
3,19:39:30,T1,1.0,X=27.7,Y=-193.8,27-04.0550 S,109-10.0272 W,Z=18.7,HR=90.2,BRG=179.1,SR=92.1
4,19:39:36,T1,1.0,X=26.9,Y=-190.8,27-04.0554 S,109-10.0254 W,Z=19.0,HR=93.2,BRG=179.4,SR=95.1
...,...,...,...,...,...,...,...,...,...,...,...
5701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
#Converting all of the files time and date columns to a single datetime object for each dataframe in the following lines.
Hab37['date'] = Hab37['DATE'] + str(' ') + Hab37['TIME']

In [172]:
Hab37['date'] = pd.to_datetime(Hab37['date'], format='%m/%d/%Y %H:%M:%S')

In [173]:
Hab39['date'] = Hab39['DATE'] + str(' ') + Hab39['TIME']

In [174]:
Hab39['date'] = pd.to_datetime(Hab39['date'], format='%m/%d/%Y %H:%M:%S')

In [175]:
CTD37['date'] = CTD37['DATE'] + str(' ') + CTD37['TIME']

In [176]:
CTD37['date'] = pd.to_datetime(CTD37['date'], format='%m/%d/%Y %H:%M:%S')

In [177]:
CTD39['date'] = CTD39['DATE'] + str(' ') + CTD39['TIME']

In [178]:
CTD39['date'] = pd.to_datetime(CTD39['date'], format='%m/%d/%Y %H:%M:%S')

In [179]:
DT37['date'] = str('2/13/2019') + str(' ') + DT37['date']

In [180]:
DT37['date'] = pd.to_datetime(DT37['date'], format='%m/%d/%Y %H:%M:%S')

In [181]:
DT39['date'] = str('2/16/2019') + str(' ') + DT39['date']

In [182]:
DT39['date'] = pd.to_datetime(DT39['date'], format='%m/%d/%Y %H:%M:%S')

In [223]:
#The latitude and longitude files are in degrees and minutes, so I converted them to degrees only. To do this I first split the data into two columns using the hyphen separating the degrees and minutes. This created a new dataframe 'split,' and I did this for both latitude and longitude.
split = DT37['Latitude'].str.split('-', n = 1, expand = True)
split_lon = DT37['Longitude'].str.split('-', n = 1, expand = True)

In [224]:
#Then used str.rstrip() to remove the remaining non-numerical characters from the string.
split[1] = split[1].str.rstrip(' ,S')
split_lon[1] = split_lon[1].str.rstrip(' ,W')

In [225]:
#And then converted the columns to integers and floats as needed for the calculations.
split[1] = split[1].astype(float)
split[1]

0        9.8848
1        9.8871
2       11.3013
3       11.2994
4       11.2989
         ...   
1108    10.8641
1109    10.8634
1110    10.8642
1111    10.8628
1112    10.8616
Name: 1, Length: 1113, dtype: float64

In [226]:
split[0] = split[0].astype(int)
split[0]

0       26
1       26
2       26
3       26
4       26
        ..
1108    26
1109    26
1110    26
1111    26
1112    26
Name: 0, Length: 1113, dtype: int32

In [227]:
#Calculating the latitude and adding it back to the original dataframe as a new column.
DT37['LAT'] = -1*((split[0])+((split[1])/60))

In [228]:
#Looking at the new column.
DT37['LAT']

0      -26.164747
1      -26.164785
2      -26.188355
3      -26.188323
4      -26.188315
          ...    
1108   -26.181068
1109   -26.181057
1110   -26.181070
1111   -26.181047
1112   -26.181027
Name: LAT, Length: 1113, dtype: float64

In [229]:
#And then dropping the NaN values.
DT37['LAT'].dropna()

0      -26.164747
1      -26.164785
2      -26.188355
3      -26.188323
4      -26.188315
          ...    
1108   -26.181068
1109   -26.181057
1110   -26.181070
1111   -26.181047
1112   -26.181027
Name: LAT, Length: 1113, dtype: float64

In [230]:
#And nowdoing the same for longitude as well.
split_lon[1] = split_lon[1].astype(float)
split_lon[1]

0       58.7199
1       58.5475
2       59.8867
3       59.8852
4       59.8843
         ...   
1108    58.5303
1109    58.5293
1110    58.5279
1111    58.5288
1112    58.5280
Name: 1, Length: 1113, dtype: float64

In [231]:
split_lon[0] = split_lon[0].astype(int)
split_lon[0]

0       102
1       102
2       102
3       102
4       102
       ... 
1108    102
1109    102
1110    102
1111    102
1112    102
Name: 0, Length: 1113, dtype: int32

In [233]:
#Calculating the longitude and adding it back to the original dataframe as a new column.
DT37['LON'] = -1*((split_lon[0])+((split_lon[1])/60))

In [234]:
DT37['LON']

0      -102.978665
1      -102.975792
2      -102.998112
3      -102.998087
4      -102.998072
           ...    
1108   -102.975505
1109   -102.975488
1110   -102.975465
1111   -102.975480
1112   -102.975467
Name: LON, Length: 1113, dtype: float64

In [235]:
#Following the same steps again for DT39.
split = DT39['Latitude'].str.split('-', n = 1, expand = True)
split_lon = DT39['Longitude'].str.split('-', n = 1, expand = True)

In [236]:
split[1] = split[1].str.rstrip(' ,S')
split_lon[1] = split_lon[1].str.rstrip(' ,W')

In [237]:
split[1] = split[1].astype(float)
split[1]

0       4.0653
1       4.0542
2       4.0547
3       4.0550
4       4.0554
         ...  
1630    3.8400
1631    3.8387
1632    3.8385
1633    3.8374
1634    3.8364
Name: 1, Length: 1635, dtype: float64

In [238]:
split[0] = split[0].astype(float)
split[0]

0       27.0
1       27.0
2       27.0
3       27.0
4       27.0
        ... 
1630    27.0
1631    27.0
1632    27.0
1633    27.0
1634    27.0
Name: 0, Length: 1635, dtype: float64

In [239]:
DT39['LAT'] = -1*((split[0])+((split[1])/60))

<ipython-input-239-30b418316132>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DT39['LAT'] = -1*((split[0])+((split[1])/60))


In [240]:
DT39['LAT'] = DT39['LAT'].dropna()

<ipython-input-240-250d893d7abe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DT39['LAT'] = DT39['LAT'].dropna()


In [241]:
DT39['LAT']

0      -27.067755
1      -27.067570
2      -27.067578
3      -27.067583
4      -27.067590
          ...    
1630   -27.064000
1631   -27.063978
1632   -27.063975
1633   -27.063957
1634   -27.063940
Name: LAT, Length: 1635, dtype: float64

In [242]:
split_lon[1] = split_lon[1].astype(float)
split_lon[1]

0       10.0314
1       10.0274
2       10.0279
3       10.0272
4       10.0254
         ...   
1630    12.2095
1631    12.2114
1632    12.2126
1633    12.2135
1634    12.2158
Name: 1, Length: 1635, dtype: float64

In [243]:
split_lon[0] = split_lon[0].astype(int)
split_lon[0]

0       109
1       109
2       109
3       109
4       109
       ... 
1630    109
1631    109
1632    109
1633    109
1634    109
Name: 0, Length: 1635, dtype: int32

In [251]:
#Calculating the longitude and adding it back to the original dataframe as a new column.
DT39['LON'] = -1*((split_lon[0])+((split_lon[1])/60))

<ipython-input-251-55205764f532>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DT39['LON'] = -1*((split_lon[0])+((split_lon[1])/60))


In [245]:
DT39['LON']

0      -109.167190
1      -109.167123
2      -109.167132
3      -109.167120
4      -109.167090
           ...    
1630   -109.203492
1631   -109.203523
1632   -109.203543
1633   -109.203558
1634   -109.203597
Name: LON, Length: 1635, dtype: float64

In [246]:
#A look at the entire dataframe, now with the LAT and LON columns which can be mapped!
DT37

,date,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Latitude,Longitude,Depth,Unnamed: 8,Unnamed: 9,Unnamed: 10,LAT,LON
0,2019-02-13 19:41:54,T1,1,X=2059.2,Y=1299.8,26-09.8848 S,102-58.7199 W,Z=1467.1,HR=3189.6,BRG=341.5,SR=3510.9,-26.164747,-102.978665
1,2019-02-13 19:42:30,T1,1,X=2054.9,Y=1587.1,26-09.8871 S,102-58.5475 W,Z=1077.5,HR=3355.3,BRG=345.2,SR=3524.1,-26.164785,-102.975792
2,2019-02-13 19:43:18,T1,1,X=-556.4,Y=-644.3,26-11.3013 S,102-59.8867 W,Z=23.4,HR=93.4,BRG=178.1,SR=96.3,-26.188355,-102.998112
3,2019-02-13 19:43:30,T1,1,X=-552.9,Y=-641.7,26-11.2994 S,102-59.8852 W,Z=23.6,HR=93.3,BRG=178.4,SR=96.2,-26.188323,-102.998087
4,2019-02-13 19:43:36,T1,1,X=-552.0,Y=-640.2,26-11.2989 S,102-59.8843 W,Z=26.0,HR=93.4,BRG=178.4,SR=96.9,-26.188315,-102.998072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,2019-02-13 21:52:06,T1,1,X=250.8,Y=1615.6,26-10.8641 S,102-58.5303 W,Z=27.8,HR=91.4,BRG=179.2,SR=95.5,-26.181068,-102.975505
1109,2019-02-13 21:52:12,T1,1,X=252.2,Y=1617.2,26-10.8634 S,102-58.5293 W,Z=29.8,HR=91.3,BRG=178.8,SR=96.1,-26.181057,-102.975488
1110,2019-02-13 21:52:18,T1,1,X=250.7,Y=1619.5,26-10.8642 S,102-58.5279 W,Z=27.2,HR=91.5,BRG=177.5,SR=95.5,-26.181070,-102.975465
1111,2019-02-13 21:52:24,T1,1,X=253.2,Y=1618.1,26-10.8628 S,102-58.5288 W,Z=26.0,HR=93.7,BRG=179.0,SR=97.2,-26.181047,-102.975480


In [247]:
DT39

,date,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Latitude,Longitude,Depth,Unnamed: 8,Unnamed: 9,Unnamed: 10,LAT,LON
0,2019-02-16 19:36:42,T1,1.0,X=8.6,Y=-200.7,27-04.0653 S,109-10.0314 W,Z=22.6,HR=88.0,BRG=178.9,SR=90.8,-27.067755,-109.167190
1,2019-02-16 19:39:12,T1,1.0,X=29.1,Y=-194.1,27-04.0542 S,109-10.0274 W,Z=11.6,HR=90.2,BRG=178.7,SR=91.0,-27.067570,-109.167123
2,2019-02-16 19:39:24,T1,1.0,X=28.3,Y=-194.9,27-04.0547 S,109-10.0279 W,Z=23.9,HR=89.1,BRG=179.0,SR=92.2,-27.067578,-109.167132
3,2019-02-16 19:39:30,T1,1.0,X=27.7,Y=-193.8,27-04.0550 S,109-10.0272 W,Z=18.7,HR=90.2,BRG=179.1,SR=92.1,-27.067583,-109.167120
4,2019-02-16 19:39:36,T1,1.0,X=26.9,Y=-190.8,27-04.0554 S,109-10.0254 W,Z=19.0,HR=93.2,BRG=179.4,SR=95.1,-27.067590,-109.167090
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,2019-02-16 22:45:42,T1,1.0,X=424.1,Y=-3801.7,27-03.8400 S,109-12.2095 W,Z=46.1,HR=90.9,BRG=180.3,SR=101.9,-27.064000,-109.203492
1631,2019-02-16 22:45:48,T1,1.0,X=426.6,Y=-3805.0,27-03.8387 S,109-12.2114 W,Z=40.5,HR=89.4,BRG=179.0,SR=98.1,-27.063978,-109.203523
1632,2019-02-16 22:45:54,T1,1.0,X=426.9,Y=-3806.9,27-03.8385 S,109-12.2126 W,Z=33.9,HR=91.2,BRG=179.6,SR=97.3,-27.063975,-109.203543
1633,2019-02-16 22:46:00,T1,1.0,X=428.9,Y=-3808.4,27-03.8374 S,109-12.2135 W,Z=29.9,HR=92.3,BRG=178.9,SR=97.0,-27.063957,-109.203558


In [207]:
#Example code from online on how to find the nearest datetime object
#dt = pd.to_datetime(“2016–04–23 11:00:00”)
#df.index.get_loc(dt, method=“nearest”)
##get index date
#idx = df.index[df.index.get_loc(dt, method='nearest')]
##row to series
#s = df.iloc[df.index.get_loc(dt, method='nearest')]

DT37_0 = (DT37['date'][0])[DT37['date'][0].index.get_loc(DT37['date'][0], method='nearest')]

AttributeError: 'Timestamp' object has no attribute 'index'